# Case Study

## Setup

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Get raw Data
df = pd.read_csv("data/house_data_training.csv", sep=';') 
#Deleting row index
df = df.iloc[:, 1:]

## Visualisierung 1.0

## Ausreißer

## Missing Values

## Noise

## Transformation

### Normalisierung

### Standardisierung

## Korrelation

## Feature Selection